#  Interactive Visual Analytics with Folium


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives




*  Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*  Calculate the distances between a launch site to its proximities



Let's first import required Python packages for this lab:


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
##  Mark all launch sites on a map
launch_sites = [
    {"name": "CCAFS SLC 40", "lat": 28.5618571, "lon": -80.577366},
    {"name": "KSC LC 39A", "lat": 28.6080585, "lon": -80.6039558},
    {"name": "VAFB SLC 4E", "lat": 34.632093, "lon": -120.610829}
]



In [4]:
launch_map = folium.Map(
    location=[30, -90],   # center of the US
    zoom_start=4
)


In [5]:
for site in launch_sites:
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=site["name"],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(launch_map)


In [6]:
launch_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [8]:
##Download and read spacex_launch_geo.csv file

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

spacex_df = pd.read_csv(url)

spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [13]:
# Houston location
houston_lat, houston_long = 29.7604, -95.3698

site_map = folium.Map(
    location=[houston_lat, houston_long],
    zoom_start=5
)

# Small yellow circle near Houston
folium.Circle(
    [houston_lat, houston_long],
    radius=10000,
    color='yellow',
    fill=True,
    fill_color='yellow'
).add_to(site_map)

site_map


In [17]:
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()
launch_sites


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
26,VAFB SLC-4E,34.632834,-120.610745
36,KSC LC-39A,28.573255,-80.646895
49,CCAFS SLC-40,28.563197,-80.576820


In [18]:
for index, row in launch_sites.iterrows():
    
    # Add circle
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(site_map)
    
    # Add marker
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(site_map)

site_map


In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The map visualization clearly shows that launch success rates improve over time across all launch sites. Most failures occurred during early launches, while later launches demonstrate higher reliability, indicating technological maturity and operational learning.

In [19]:
#  Mark the success/failed launches for each site on the map

# Mark success and failed launches
for _, row in spacex_df.iterrows():
    
    # Set marker color based on class
    if row['class'] == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'
    
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=4,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7
    ).add_to(site_map)


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [21]:
marker_cluster = MarkerCluster()


* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [22]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

spacex_df[['class', 'marker_color']].head()


,class,marker_color
0,0,red
1,0,red
2,0,red
3,0,red
4,0,red


* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [23]:
# Add marker_cluster to site_map
site_map.add_child(marker_cluster)

# Add a marker for each launch result
for index, record in spacex_df.iterrows():
    
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(
            icon='info-sign',
            color='white',
            icon_color=record['marker_color']
        ),
        popup=record['Launch Site']
    )
    
    marker_cluster.add_child(marker)

site_map


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [25]:
#  Calculate the distances between a launch site to its proximities
from math import sin, cos, sqrt, atan2, radians



In [26]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth radius in kilometers
    R = 6371.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [27]:
equator_lat = 0
equator_lon = 0


In [28]:
coast_lat = 28.563197
coast_lon = -80.576820


In [29]:
for _, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    lat = row['Lat']
    lon = row['Long']

    dist_equator = calculate_distance(lat, lon, equator_lat, equator_lon)
    dist_coast = calculate_distance(lat, lon, coast_lat, coast_lon)

    print(f"Launch Site: {site_name}")
    print(f"Distance to Equator: {dist_equator:.2f} km")
    print(f"Distance to Coast: {dist_coast:.2f} km\n")


Launch Site: CCAFS LC-40
Distance to Equator: 9088.26 km
Distance to Coast: 0.11 km

Launch Site: CCAFS SLC-40
Distance to Equator: 9088.22 km
Distance to Coast: 0.00 km

Launch Site: KSC LC-39A
Distance to Equator: 9095.13 km
Distance to Coast: 6.93 km

Launch Site: VAFB SLC-4E
Distance to Equator: 12761.85 km
Distance to Coast: 3825.85 km



Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [30]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [32]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [33]:
# Launch site coordinate (example: CCAFS SLC-40)
launch_site_lat = 28.5618571
launch_site_lon = -80.577366

launch_site_coordinate = [launch_site_lat, launch_site_lon]

# Closest coastline coordinate (given by lab / visually selected)
coastline_lat = 28.56367
coastline_lon = -80.57163

coastline_coordinate = [coastline_lat, coastline_lon]


In [34]:
distance = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    coastline_lat,
    coastline_lon
)


In [35]:
from folium.features import DivIcon

distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance)
    )
)

site_map.add_child(distance_marker)


In [36]:
lines = folium.PolyLine(
    locations=[launch_site_coordinate, coastline_coordinate],
    weight=2,
    color='red'
)

site_map.add_child(lines)


In [37]:
site_map


In [38]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
   )

* Draw a `PolyLine` between a launch site to the selected coastline point


In [39]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [40]:
# Launch site (CCAFS SLC-40)
launch_site_lat = 28.5618571
launch_site_lon = -80.577366
launch_site_coordinate = [launch_site_lat, launch_site_lon]

# Closest city (example: Cape Canaveral city)
city_lat = 28.3922
city_lon = -80.6077
city_coordinate = [city_lat, city_lon]


In [41]:
distance_city = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    city_lat,
    city_lon
)


In [42]:
from folium.features import DivIcon

city_distance_marker = folium.Marker(
    city_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#2c3e50;"><b>{:.2f} KM</b></div>'.format(distance_city)
    )
)

site_map.add_child(city_distance_marker)


In [43]:
city_line = folium.PolyLine(
    locations=[launch_site_coordinate, city_coordinate],
    weight=2,
    color='blue'
)

site_map.add_child(city_line)


In [44]:
site_map
